# Auswertung Adliswil

# 1. Vorbereitung

Zuerst importiere ich die nötigen Libraries.

In [1]:
import pandas as pd

Ich setze die Option, dass ich später in den Dataframes alle Angaben sehe.

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

Danach importiere ich die beiden Listen, die ich brauche. Jene mit den Vorstössen und jene mit den Namen und Parteien der Politiker/innen.

In [3]:
path = "All_Datas/daten_adliswil.csv"
df= pd.read_csv(path)

In [4]:
path2 = "All_Datas/Parlamentarier_Adliswil.csv"
df_parlamentarier = pd.read_csv(path2)

Damit ich die beiden Listen später mergen kann, muss ich aus den Listen in der Spalte Namen-neu des Vorstoss-Dataframes die einzelnen Elemente herausziehen. Ich will also eine separate Zeile für jede Person, die an einem Vorstoss beteilitgt war. Dazu mache ich eine Kopie des Dataframes, um nichts zu überschreiben. Danach zerteile ich mit der lambda-Funktion die Liste in einzelne Elemente. Und zwar immer dann, wenn ein Komma kommt. Mit der Funktion .explode teile ich die definierten Namen in der Liste schliesslich in einzelne Zeilen auf.

In [17]:
df.head(3)

,Titel,Geschaeftsart,Status,Datum,Namen,Namen-neu,ID-Vorstoss
0,Städtische Beteiligung an Unternehmungen,Postulat,Abgeschrieben,2014-11-21,[' Senn Mario'],['Senn Mario'],1
1,ZKB Jubiläums-Dividende,Interpellation,Beantwortet,2019-11-06,"[' Mäder Gabriel', ' Schanz Simon', '', 'Stefan Neubert (Mitunterzeichner/in)\n Anke Würl (Mitunterzeichner/in)\n Martin Rüttimann (Mitunterzeichner/in)\n Walter Uebersax (Erstunterzeichner/in)', '', '', '']","['Mäder Gabriel', 'Schanz Simon', 'Stefan Neubert', 'Anke Würl', 'Martin Rüttimann', 'Walter Uebersax']",2
2,Zwischennutzung Stadthausareal,Postulat,Überwiesen,2011-09-07,[' Clesle Hanspeter'],['Clesle Hanspeter'],3


In [18]:
df_vorstoesse = df.copy()

df_vorstoesse['Namen-neu'] = df_vorstoesse['Namen-neu'].apply(lambda x: x if pd.isna(x) else x.split(','))
df_vorstoesse = df_vorstoesse.explode('Namen-neu')
df_vorstoesse.head(3)

,Titel,Geschaeftsart,Status,Datum,Namen,Namen-neu,ID-Vorstoss
0,Städtische Beteiligung an Unternehmungen,Postulat,Abgeschrieben,2014-11-21,[' Senn Mario'],['Senn Mario'],1
1,ZKB Jubiläums-Dividende,Interpellation,Beantwortet,2019-11-06,"[' Mäder Gabriel', ' Schanz Simon', '', 'Stefan Neubert (Mitunterzeichner/in)\n Anke Würl (Mitunterzeichner/in)\n Martin Rüttimann (Mitunterzeichner/in)\n Walter Uebersax (Erstunterzeichner/in)', '', '', '']",['Mäder Gabriel',2
1,ZKB Jubiläums-Dividende,Interpellation,Beantwortet,2019-11-06,"[' Mäder Gabriel', ' Schanz Simon', '', 'Stefan Neubert (Mitunterzeichner/in)\n Anke Würl (Mitunterzeichner/in)\n Martin Rüttimann (Mitunterzeichner/in)\n Walter Uebersax (Erstunterzeichner/in)', '', '', '']",'Schanz Simon',2


Nun habe ich für alle Personen, die an einem Vorstoss beteiligt waren, eine einzelne Zeile. Um sie mit der Namensliste mergen zu können, müssen die Namen identisch sein. Ich muss also noch sämtliche Klammern und Anführungszeichen eliminieren. Das mache ich mit Hilfe von Regex.

In [19]:
df_vorstoesse['Namen-neu'] = df_vorstoesse['Namen-neu'].str.replace(r"[\[\]']|''", '').str.strip()
df_vorstoesse.head(3)

/var/folders/0_/g6gf3kvd4wd5y2c_ls8l_k280000gn/T/ipykernel_32044/273107438.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_vorstoesse['Namen-neu'] = df_vorstoesse['Namen-neu'].str.replace(r"[\[\]']|''", '').str.strip()


,Titel,Geschaeftsart,Status,Datum,Namen,Namen-neu,ID-Vorstoss
0,Städtische Beteiligung an Unternehmungen,Postulat,Abgeschrieben,2014-11-21,[' Senn Mario'],Senn Mario,1
1,ZKB Jubiläums-Dividende,Interpellation,Beantwortet,2019-11-06,"[' Mäder Gabriel', ' Schanz Simon', '', 'Stefan Neubert (Mitunterzeichner/in)\n Anke Würl (Mitunterzeichner/in)\n Martin Rüttimann (Mitunterzeichner/in)\n Walter Uebersax (Erstunterzeichner/in)', '', '', '']",Mäder Gabriel,2
1,ZKB Jubiläums-Dividende,Interpellation,Beantwortet,2019-11-06,"[' Mäder Gabriel', ' Schanz Simon', '', 'Stefan Neubert (Mitunterzeichner/in)\n Anke Würl (Mitunterzeichner/in)\n Martin Rüttimann (Mitunterzeichner/in)\n Walter Uebersax (Erstunterzeichner/in)', '', '', '']",Schanz Simon,2


Als nächstes will ich die beiden Listen so zusammenfügen, dass ich mit den IDs der Politiker und der Vorstösse arbeiten kann. So kann ich verhindern, dass sich Fehler einschleichen, falls zwei Politiker/innen gleich heissen. Ich merge die beiden Listen zur df_ID

In [20]:
df_ID = df_vorstoesse.merge(df_parlamentarier, left_on='Namen-neu', right_on='Name', how='left')[['ID-Vorstoss', 'ID-Politiker']]

In [21]:
df_ID.head(5)

,ID-Vorstoss,ID-Politiker
0,1,91.0
1,2,62.0
2,2,86.0
3,2,96.0
4,2,6.0


Jetzt habe ich die Grundlage geschaffen, um mit den konkreten Auswertungen beginnen zu konnen.

## 1. Welche Parteien reichen die häufigsten Vorstösse ein?